### Note

In this note, I'll do the following experiment.

1. Get committee.Here we choose SB1 committee.
2. Get adversarial samples: CWL2 and PGD
3. For each sample, get mask of dataset at which committee disagree. Values are 0, 0.24, 0.3, 0.45, 0.6. We select 0.4 as the entropy threshold so that new labels are $0,\ldots,19$.
4. $D = (X,Y) = (X_0, Y_0) \sqcup (X_1, Y_1)$ where $X_0 = \{I(x)<0.4\}, X_1 = \{I(x) \geq 0.4\},$ and  $Y_1 = \{y+10 :  x \in X_1\}$. See the accuracy on $X_0$ and $X_1$ of Committee.


In [1]:
# import necessary libraries and set experiments.

# load necessary libraries
import os, glob
import numpy as np
import tensorflow as tf
import pandas as pd
import keras
from keras.models import load_model
from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras import Model
from scipy.stats import entropy
import math
loaded_models = {}
# load additional models from another folder
# assuming these models are also saved in Keras format with '.keras' extension
# Say 'SB-type', where 'SB' stands for 'Specialized Base'.
folder = 'CIFAR10models/more_tunned/'
pattern = os.path.join(folder, '*_more_specialized*.keras')
file_list = sorted(glob.glob(pattern))
loaded_models.update({os.path.basename(f): load_model(f) for f in file_list})

label_dict = {(3,1) : 'ResNet20v1',
        (3,2) : 'ResNet20v2',
        (9,1): 'ResNet56v1',
        (9,2): 'ResNet56v2'}

(x, y), (x_test, y_test) = cifar10.load_data()
x_train, x_v, y_train, y_v = train_test_split(x, y, test_size=0.2, 
                                              random_state=42) # random state has been always 42.
x_train = x_train.astype('float32') / 255.0
x_val = x_v.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train_categorical_10 = to_categorical(y_train,10)
y_val_categorical_10 = to_categorical(y_v,10)
y_test_categorical_10 = to_categorical(y_test,10)
y_test_categorical_20 = to_categorical(y_test,20)

# Set the predetermined random seed
rng = np.random.default_rng(seed=42)

# Define ranges
range1 = [range(8), range(4), range(4), range(4)]
range2 = [range(8), range(4), [0], range(4)]

# Sample one tuple from each
tuple1 = tuple(rng.choice(r) for r in range1)

print("Comm 1:", tuple1)
comm_SB_1 = []
for file_name, model in loaded_models.items():
    # Extract model name from the file name
    base = file_name.replace('.keras','')
    parts = base.split('_')
    if parts[0] == 'n':
        model_name = label_dict[(int(parts[1]),int(parts[2][-1]))] + '_B_'+parts[-1][-1]
    else:
        model_name = parts[0] + '_SB_' + parts[-1]
    # print(f"Model: {model_name}")
    parts = model_name.split('_')
    if parts[0] == 'ResNet20v1':
        if parts[1] == 'SB':
            if int(parts[-1]) == tuple1[0]:
                comm_SB_1.append((model_name,model))
    if parts[0] == 'ResNet20v2':
        if parts[1] == 'SB':
            if int(parts[-1]) == tuple1[1]:
                comm_SB_1.append((model_name,model))
    if parts[0] == 'ResNet56v1':
        if parts[1] == 'SB':
            if int(parts[-1]) == tuple1[2]:
                comm_SB_1.append((model_name,model))
    if parts[0] == 'ResNet56v2':
        if parts[1] == 'SB':
            if int(parts[-1]) == tuple1[3]:
                comm_SB_1.append((model_name,model))
                
print('comm_SB_1', comm_SB_1)


# As SB model shows better performance on the certain part, let use SB type models to analyze the behavior on the certain part against adversarial examples.

# Load the adversarial examples

## set all files.
## Possibly there is no merged testing dataset, or validation dataset.
## We only look for testing dataset.
import re

pattern1 = re.compile(r'_\d+to\d+_')
pattern2 = re.compile(r'test')

folder = './adversarial_examples/gen_by_ResNet'
all_files = os.listdir(folder)

filtered_files = [
    fname for fname in all_files
    if fname.endswith('.npy') and not pattern1.search(fname) and not pattern2.search(fname)
]


## load all the training and validation dataset

loaded_training_dataset = {}
loaded_validation_dataset = {}
import re
for f in filtered_files:
    
    base = f.replace('.keras.npy','').replace('.npy','')
    parts = base.split('_')
    '''
    cwl2_x_tr_untargeted
    cwl2_x_v_untargeted.npy
    pgd_0.376_x_target_to_ll.npy
    pgd_0.376_x_target_to_second.npy
    pgd_0.376_x_untarget.npy
    pgd_0.376_x_val_target_to_ll.npy
    pgd_0.376_x_val_target_to_second.npy
    pgd_0.376_x_val_untarget.npy
    '''
    
    attack_type = parts[0]
    if attack_type == 'cwl2':
        direction = 'untargeted'
        if parts[2] == 'tr':
            train_or_val = 'train'
        else:
            train_or_val = 'val'
    else:
        attack_type = 'PGD by ResNet56v1'
        direction = parts[-1]
        if direction == 'll':
            direction = 'targeted to least'
        elif direction =='second':
            direction = 'targeted to 2nd'
        else:
            direction = 'untargeted'
        if parts[3] == 'val':
            train_or_val = 'val'
        else:
            train_or_val = 'train'
    key = (attack_type, direction, train_or_val)
    if train_or_val == 'train':
        loaded_training_dataset[key] = np.load(os.path.join(folder, f))
    else:
        loaded_validation_dataset[key] = np.load(os.path.join(folder, f))

print('-'*50, 'keys for adv samples', '-'*50)
for k in loaded_training_dataset.keys():
    print(k)
    
for k in loaded_validation_dataset.keys():
    print(k)
    
    
def get_entropy_array(committee : list[tuple[str,Model]], sample : np.ndarray): 
    '''
    param committee: list of tuple (member, model)
    param sample: numpy array of (n,) shape
    
    return tuple of dict, numpy array of (n,) shape
    dictionary : committee member -> list of [answer to the sample, certainty to the sample]
    '''
    comm_pred = {}
    for member, model in committee:
        p = model.predict(sample,verbose = 0)
        comm_pred[member] = [
                                np.argmax(p,axis=-1),
                                np.max(p,axis=-1)
                                ]
    stacked = np.stack([member_pred[0] for member_pred in comm_pred.values()], axis=1)
    counts = np.apply_along_axis(lambda x: np.bincount(x, minlength=10), axis=1, arr=stacked)
    probs = counts / counts.sum(axis=1, keepdims=True)
    disagreements = entropy(probs, axis=1, base = 10)
    
    return disagreements

committee = comm_SB_1

## CWL2 train and val
cwl2_train = loaded_training_dataset[('cwl2', 'untargeted', 'train')]
cwl2_val = loaded_validation_dataset[('cwl2', 'untargeted', 'val')]
entropy_cwl2_train = np.load('./entropies/entropy_SB1_cwl2_untargeted_train.npy')
entropy_cwl2_val = np.load('./entropies/entropy_SB1_cwl2_untargeted_val.npy')
cwl2_new_label_train = np.load('./new_labels/SB1/new_label_cwl2_untargeted_train.npy')
cwl2_new_label_val= np.load('./new_labels/SB1/new_label_cwl2_untargeted_val.npy')
cwl2_new_label_test= np.load('./new_labels/SB1/new_label_cwl2_untargeted_test.npy')
## PGD untargeted train and val
PGD_train = loaded_training_dataset[('PGD by ResNet56v1', 'untargeted', 'train')]
PGD_val = loaded_validation_dataset[('PGD by ResNet56v1', 'untargeted', 'val')]
entropy_PGD_untargeted_train = np.load('./entropies/entropy_SB1_PGD by ResNet56v1_untargeted_train.npy')
entropy_PGD_untargeted_val = np.load('./entropies/entropy_SB1_PGD by ResNet56v1_untargeted_val.npy')
PGD_untargeted_new_label_train = np.load('./new_labels/SB1/new_label_PGD_untargeted_train.npy')
PGD_untargeted_new_label_val = np.load('./new_labels/SB1/new_label_PGD_untargeted_val.npy')
PGD_untargeted_new_label_test = np.load('./new_labels/SB1/new_label_PGD_untargeted_test.npy')
## PGD targeted train and val
PGD_targeted_train = loaded_training_dataset[('PGD by ResNet56v1', 'targeted to least', 'train')]
PGD_targeted_val = loaded_validation_dataset[('PGD by ResNet56v1', 'targeted to least', 'val')]
entropy_PGD_targeted_train = np.load('./entropies/entropy_SB1_PGD by ResNet56v1_targeted to least_train.npy')
entropy_PGD_targeted_val = np.load('./entropies/entropy_SB1_PGD by ResNet56v1_targeted to least_val.npy')
PGD_targeted_new_label_train = np.load('./new_labels/SB1/new_label_PGD_targeted_train.npy')
PGD_targeted_new_label_val = np.load('./new_labels/SB1/new_label_PGD_targeted_val.npy')
PGD_targeted_new_label_test = np.load('./new_labels/SB1/new_label_PGD_targeted_test.npy')

def committee_answers(committee, sample, entropy, mask = None):
    if mask is None:
        # Create a mask for non-zero entropy values
        mask = entropy != 0

    # Prepare data for the table
    table_data = {
        'Original Label': y_train[:sample.shape[0]][mask].reshape(-1),
        'Entropy': np.round(entropy[mask], 3)
    }

    # Loop through the committee to get predictions
    for member, model in committee:
        # Predictions on CWL2 adversarial data
        p  = model.predict(sample[mask], verbose=0)
        table_data[f'{member[-1]}p'] = np.argmax(p , axis=-1)
        table_data[f'{member[-1]}c'] = np.round(np.max(p , axis=-1), 3)

    # Create and display the DataFrame
    df = pd.DataFrame(table_data)
    return df



Comm 1: (0, 3, 2, 1)
comm_SB_1 [('ResNet20v1_SB_0', <keras.engine.functional.Functional object at 0x1479dfad9fc0>), ('ResNet20v2_SB_3', <keras.engine.functional.Functional object at 0x1479d2982ec0>), ('ResNet56v1_SB_2', <keras.engine.functional.Functional object at 0x1479d2b25f60>), ('ResNet56v2_SB_1', <keras.engine.functional.Functional object at 0x1479df51a320>)]
-------------------------------------------------- keys for adv samples --------------------------------------------------
('cwl2', 'untargeted', 'train')
('PGD by ResNet56v1', 'untargeted', 'train')
('PGD by ResNet56v1', 'targeted to 2nd', 'train')
('PGD by ResNet56v1', 'targeted to least', 'train')
('PGD by ResNet56v1', 'targeted to least', 'val')
('cwl2', 'untargeted', 'val')
('PGD by ResNet56v1', 'targeted to 2nd', 'val')
('PGD by ResNet56v1', 'untargeted', 'val')


In [8]:
y_samples = [y_train.reshape(-1), y_v.reshape(-1), y_test.reshape(-1)]
name = ['training','validation','testing']
## Original
for i, sample in enumerate([x_train, x_val, x_test]):
    print(f'On Original {name[i]} samples...')
    entropy_array = get_entropy_array(committee,sample)
    mask = entropy_array>0.4
    
    y = y_samples[i]
    for member, model in comm_SB_1:
        
        p = model.predict(sample,verbose =0)
        a = np.argmax(p,axis=-1)
        c = np.max(p, axis=-1)
        print(f'Member: {member}, Acc on Whole: {np.sum(a == y)/len(y)}, on X_0: {np.sum(a[~mask] == y[~mask])/len(y[~mask])},', end=' ')
        if np.sum(mask) == 0:
            print('Empty X_1')
        else:
            print(f'on X_1: {np.sum(a[mask] == y[mask])/len(y[mask])}') 
        


## CWL2
entropy_array = get_entropy_array(committee,np.load('./adversarial_examples/gen_by_ResNet/cwl2_x_test_untargeted.npy'))
entropies = [entropy_cwl2_train,entropy_cwl2_val,entropy_array]

for i, sample in enumerate([cwl2_train, cwl2_val, np.load('./adversarial_examples/gen_by_ResNet/cwl2_x_test_untargeted.npy')]):
    print(f'On CWL2 {name[i]} samples...')
    if i == 0 :
        size = 10001
    else: 
        size = 10000
    mask = entropies[i] > 0.4
    y = y_samples[i][:size]
    for member, model in comm_SB_1:
        p = model.predict(sample[:size],verbose =0)
        a = np.argmax(p,axis=-1)
        c = np.max(p, axis=-1)
        print(f'Member: {member}, Acc on Whole: {np.sum(a == y)/len(y)}, on X_0: {np.sum(a[~mask] == y[~mask])/len(y[~mask])},', end=' ')
        if np.sum(mask) == 0:
            print('Empty X_1')
        else:
            print(f'on X_1: {np.sum(a[mask] == y[mask])/len(y[mask])}') 
        

## PGD untargeted
entropy_array = get_entropy_array(committee,np.load('./adversarial_examples/gen_by_ResNet/pgd_0.376_x_test_untarget_gen_by_n_9_v1_cifar10.keras.npy'))
entropies = [entropy_PGD_untargeted_train,entropy_PGD_untargeted_val,entropy_array]

for i, sample in enumerate([PGD_train, PGD_val, np.load('./adversarial_examples/gen_by_ResNet/pgd_0.376_x_test_untarget_gen_by_n_9_v1_cifar10.keras.npy')]):
    print(f'On PGD untargeted {name[i]} samples...')
    mask = entropies[i] > 0.4
    y = y_samples[i]
    for member, model in comm_SB_1:
        p = model.predict(sample,verbose =0)
        a = np.argmax(p,axis=-1)
        c = np.max(p, axis=-1)
        print(f'Member: {member}, Acc on Whole: {np.sum(a == y)/len(y)}, on X_0: {np.sum(a[~mask] == y[~mask])/len(y[~mask])},', end=' ')
        if np.sum(mask) == 0:
            print('Empty X_1')
        else:
            print(f'on X_1: {np.sum(a[mask] == y[mask])/len(y[mask])}') 
        

## PGD targeted

entropy_array = get_entropy_array(committee,np.load('./adversarial_examples/gen_by_ResNet/pgd_0.376_x_test_target_to_ll_gen_by_n_9_v1_cifar10.keras.npy'))
entropies = [entropy_PGD_targeted_train,entropy_PGD_targeted_val,entropy_array]
for i, sample in enumerate([PGD_targeted_train, PGD_targeted_val, np.load('./adversarial_examples/gen_by_ResNet/pgd_0.376_x_test_target_to_ll_gen_by_n_9_v1_cifar10.keras.npy')]):
    print(f'On PGD targeted {name[i]} samples...')
    mask = entropies[i] > 0.4
    y = y_samples[i]
    for member, model in comm_SB_1:
        p = model.predict(sample,verbose =0)
        a = np.argmax(p,axis=-1)
        c = np.max(p, axis=-1)
        print(f'Member: {member}, Acc on Whole: {np.sum(a == y)/len(y)}, on X_0: {np.sum(a[~mask] == y[~mask])/len(y[~mask])},', end=' ')
        if np.sum(mask) == 0:
            print('Empty X_1')
        else:
            print(f'on X_1: {np.sum(a[mask] == y[mask])/len(y[mask])}') 
        


On Original training samples...
Member: ResNet20v1_SB_0, Acc on Whole: 1.0, on X_0: 1.0, Empty X_1
Member: ResNet20v2_SB_3, Acc on Whole: 1.0, on X_0: 1.0, Empty X_1
Member: ResNet56v1_SB_2, Acc on Whole: 1.0, on X_0: 1.0, Empty X_1
Member: ResNet56v2_SB_1, Acc on Whole: 0.999975, on X_0: 0.999975, Empty X_1
On Original validation samples...
Member: ResNet20v1_SB_0, Acc on Whole: 0.9082, on X_0: 0.9271809661139149, on X_1: 0.27491408934707906
Member: ResNet20v2_SB_3, Acc on Whole: 0.9128, on X_0: 0.9305798743433927, on X_1: 0.31958762886597936
Member: ResNet56v1_SB_2, Acc on Whole: 0.9072, on X_0: 0.9242970439798125, on X_1: 0.33676975945017185
Member: ResNet56v2_SB_1, Acc on Whole: 0.9222, on X_0: 0.9373776908023483, on X_1: 0.41580756013745707
On Original testing samples...
Member: ResNet20v1_SB_0, Acc on Whole: 0.9052, on X_0: 0.9224510813594232, on X_1: 0.3275862068965517
Member: ResNet20v2_SB_3, Acc on Whole: 0.9068, on X_0: 0.925849639546859, on X_1: 0.2689655172413793
Member: Re